### Manipulation of PeaskDB de novo-assisted database search results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_July2020_QE_T0/DN50_10lgP15/121_TROCAS_QE_July2020_PEAKS_24/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_July2020_QE_T0/DN50_10lgP15/121_TROCAS_QE_July2020_PEAKS_24


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 

 121_TROCAS_QE_July2020_PEAKS_24_peptide.csv    'de novo only peptides.csv'
 121_TROCAS_QE_July2020_PEAKS_24_protein.csv     peptides.pep.xml
 121_TROCAS_QE_July2020_PEAKS_24_protein.fasta   protein-peptides.csv
'DB search psm.csv'


In [4]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup126 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_July2020_QE_T0/DN50_10lgP15/121_TROCAS_QE_July2020_PEAKS_24/121_TROCAS_QE_July2020_PEAKS_24_peptide.csv")

# remove redundant rows
peaksdb126 = pd.DataFrame.drop_duplicates(peaksdbdup126)

print(peaksdb126.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb126.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb126['#Spec']
del peaksdb126['Accession']
del peaksdb126['PTM']
del peaksdb126['AScore']

mean_length = peaksdb126['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup126))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb126))

#look at the dataframe
peaksdb126.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area 588-TROCAS7-121-NMCPM_50%_GF75com1', 'Fraction', 'Scan',
       'Source File', '#Spec', '#Spec 588-TROCAS7-121-NMCPM_50%_GF75com1',
       'Accession', 'PTM', 'AScore'],
      dtype='object')
11.514925373134329
# redundant peaksdb peptides in combined dataframe 134
# nonredundant peaksdb peptides in combined dataframe 134


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,LPQVEGTGGDVQPSQDLVR,94.33,1994.0068,19,3.7,998.0143,56.30,12700000.0,3,15739,20200714_588_TROCAS7_121_NMCPM_50%_gf75com_1_0...
1,VIGQNEAVDAVSNAIR,70.59,1654.8638,16,2.8,828.4415,69.39,10100000.0,3,19565,20200714_588_TROCAS7_121_NMCPM_50%_gf75com_1_0...
2,SSGSSYPSLLQC(+57.02)LK,61.18,1525.7446,14,2.0,763.8811,72.59,181000.0,3,20628,20200714_588_TROCAS7_121_NMCPM_50%_gf75com_1_0...
3,LGEHNIDVLEGNEQ(+.98)FINAAK,55.24,2211.0808,20,14.0,1106.5631,68.66,0.0,3,19320,20200714_588_TROCAS7_121_NMCPM_50%_gf75com_1_0...
4,VTDAEIAEVLAR,54.09,1285.6877,12,3.6,643.8535,71.74,1590000.0,3,20367,20200714_588_TROCAS7_121_NMCPM_50%_gf75com_1_0...


In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb126['A'] = peaksdb126['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb126['C'] = peaksdb126['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb126['D'] = peaksdb126['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb126['E'] = peaksdb126['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb126['F'] = peaksdb126['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb126['G'] = peaksdb126['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb126['H'] = peaksdb126['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb126 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb126['I'] = peaksdb126['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb126['K'] = peaksdb126['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb126['L'] = peaksdb126['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb126['M'] = peaksdb126['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb126['N'] = peaksdb126['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb126['P'] = peaksdb126['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb126['Q'] = peaksdb126['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb126['R'] = peaksdb126['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb126['S'] = peaksdb126['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb126['T'] = peaksdb126['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb126['V'] = peaksdb126['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb126['W'] = peaksdb126['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb126['Y'] = peaksdb126['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb126['c-carb'] = peaksdb126['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb126['m-oxid'] = peaksdb126['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb126['n-deam'] = peaksdb126['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a lamba function to enumerate the # of deamidated Q's in each peptide
peaksdb126['q-deam'] = peaksdb126['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb126['stripped_peptide'] = peaksdb126['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb126['stripped_length'] = peaksdb126['stripped_peptide'].apply(len)

peaksdb126['NAAF_num.'] = peaksdb126['Area'] / peaksdb126['stripped_length']

# total the number of modifications in sequence
peaksdb126['ptm-total'] = peaksdb126['c-carb'] + peaksdb126['m-oxid'] + peaksdb126['n-deam'] + peaksdb126['q-deam']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb126['stripped_IL']= peaksdb126['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb126.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/126_MN_T00_GF_PDB/126B_MN_T00_GF_PDB.csv")

# check out the results
peaksdb126.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,LPQVEGTGGDVQPSQDLVR,94.33,1994.0068,19,3.7,998.0143,56.30,12700000.0,3,15739,...,0,0,0,0,0,LPQVEGTGGDVQPSQDLVR,19,668421.052632,0,LPQVEGTGGDVQPSQDLVR
1,VIGQNEAVDAVSNAIR,70.59,1654.8638,16,2.8,828.4415,69.39,10100000.0,3,19565,...,0,0,0,0,0,VIGQNEAVDAVSNAIR,16,631250.000000,0,VLGQNEAVDAVSNALR
2,SSGSSYPSLLQC(+57.02)LK,61.18,1525.7446,14,2.0,763.8811,72.59,181000.0,3,20628,...,1,1,0,0,0,SSGSSYPSLLQCLK,14,12928.571429,1,SSGSSYPSLLQCLK
3,LGEHNIDVLEGNEQ(+.98)FINAAK,55.24,2211.0808,20,14.0,1106.5631,68.66,0.0,3,19320,...,0,0,0,0,1,LGEHNIDVLEGNEQFINAAK,20,0.000000,1,LGEHNLDVLEGNEQFLNAAK
4,VTDAEIAEVLAR,54.09,1285.6877,12,3.6,643.8535,71.74,1590000.0,3,20367,...,0,0,0,0,0,VTDAEIAEVLAR,12,132500.000000,0,VTDAELAEVLAR


In [6]:
# keep only stripped peptide I/L and NAAF
db_126 = peaksdb126[['stripped_IL', 'Area', 'NAAF_num.']]

db_126.set_index('stripped_IL')

# write modified dataframe to new txt file
db_126.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/126B_MN_T00_GF_PDB_ILnaafs.csv")

db_126.head()

,stripped_IL,Area,NAAF_num.
0,LPQVEGTGGDVQPSQDLVR,12700000.0,668421.052632
1,VLGQNEAVDAVSNALR,10100000.0,631250.000000
2,SSGSSYPSLLQCLK,181000.0,12928.571429
3,LGEHNLDVLEGNEQFLNAAK,0.0,0.000000
4,VTDAELAEVLAR,1590000.0,132500.000000


### Exporting txt files of stripped peptides at confidence cutoffs:

In [7]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep126moddup = peaksdb126[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep126dup = peaksdb126[["stripped_peptide"]]

# deduplicate both of these lists
pep126mod = pep126moddup.drop_duplicates()
pep126 = pep126dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 126:", len(pep126moddup))
print("Deduplicated modified PeaksDB peptides in 126:", len(pep126mod))
print("Total stripped PeaksDB peptides in 126:", len(pep126dup))
print("Deduplicated stripped PeaksDB peptides in 126:", len(pep126))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep126.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/126_MN_T00_GF_PDB/126B_MN_T00_GF_PDB_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/126_MN_T00_GF_PDB/126B_MN_T00_GF_PDB_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/126_MN_T00_GF_PDB/126B_MN_T00_GF_PDB_stripped_peptides.fas

# look at the stripped peptides
pep126.head()

Total modified PeaksDB peptides in 126: 134
Deduplicated modified PeaksDB peptides in 126: 134
Total stripped PeaksDB peptides in 126: 134
Deduplicated stripped PeaksDB peptides in 126: 127


,stripped_peptide
0,LPQVEGTGGDVQPSQDLVR
1,VIGQNEAVDAVSNAIR
2,SSGSSYPSLLQCLK
3,LGEHNIDVLEGNEQFINAAK
4,VTDAEIAEVLAR


## NAAF correction and exporting files with AA and PTM totals:

In [8]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb126['A'].sum(),
        'C': peaksdb126['C'].sum(),
        'D': peaksdb126['D'].sum(),
        'E': peaksdb126['E'].sum(),
        'F': peaksdb126['F'].sum(),
        'G': peaksdb126['G'].sum(),
        'H': peaksdb126['H'].sum(),
        'I': peaksdb126['I'].sum(),
        'K': peaksdb126['K'].sum(),
        'L': peaksdb126['L'].sum(),
        'M': peaksdb126['M'].sum(),
        'N': peaksdb126['N'].sum(),
        'P': peaksdb126['P'].sum(),
        'Q': peaksdb126['Q'].sum(),
        'R': peaksdb126['R'].sum(),
        'S': peaksdb126['S'].sum(),
        'T': peaksdb126['T'].sum(),
        'V': peaksdb126['V'].sum(),
        'W': peaksdb126['W'].sum(),
        'Y': peaksdb126['Y'].sum(),
        'c-carb': peaksdb126['c-carb'].sum(),
        'm-oxid': peaksdb126['m-oxid'].sum(),
        'n-deam': peaksdb126['n-deam'].sum(),
        'q-deam': peaksdb126['q-deam'].sum(),
        'Total area': peaksdb126['Area'].sum(),
        'Total length': peaksdb126['stripped_length'].sum()
       }

totalpeaksdb126 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 
                                              'n-deam', 'q-deam', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb126['% C w/ carb'] = totalpeaksdb126['c-carb'] / totalpeaksdb126['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb126['% M w/ oxid'] = totalpeaksdb126['m-oxid'] / totalpeaksdb126['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb126['% N w/ deam'] = totalpeaksdb126['n-deam'] / totalpeaksdb126['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb126['% Q w/ deam'] = totalpeaksdb126['q-deam'] / totalpeaksdb126['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb126['NAAF denom.'] = totalpeaksdb126['Total area'] / totalpeaksdb126['Total length']

# write modified dataframe to new txt file
totalpeaksdb126.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/126_MN_T00_GF_PDB/126B_MN_T00_GF_PDB_totals.csv")

totalpeaksdb126.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,147,8,82,79,34,132,31,91,73,144,...,5,15,13,302386040.0,1504,1.0,0.217391,0.1875,0.206349,201054.547872


In [9]:
# use the calculated NAAF factor (in totalpeaksdb126 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 201054.547872

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb126['NAAF factor'] = (peaksdb126['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksdb126_NAAF = peaksdb126[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'n-deam', 'q-deam']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb126_NAAF['A-NAAF20'] = peaksdb126_NAAF['A'] * peaksdb126['NAAF factor']
peaksdb126_NAAF['C-NAAF20'] = peaksdb126_NAAF['C'] * peaksdb126['NAAF factor']
peaksdb126_NAAF['D-NAAF20'] = peaksdb126_NAAF['D'] * peaksdb126['NAAF factor']
peaksdb126_NAAF['E-NAAF20'] = peaksdb126_NAAF['E'] * peaksdb126['NAAF factor']
peaksdb126_NAAF['F-NAAF20'] = peaksdb126_NAAF['F'] * peaksdb126['NAAF factor']
peaksdb126_NAAF['G-NAAF20'] = peaksdb126_NAAF['G'] * peaksdb126['NAAF factor']
peaksdb126_NAAF['H-NAAF20'] = peaksdb126_NAAF['H'] * peaksdb126['NAAF factor']
peaksdb126_NAAF['I-NAAF20'] = peaksdb126_NAAF['I'] * peaksdb126['NAAF factor']
peaksdb126_NAAF['K-NAAF20'] = peaksdb126_NAAF['K'] * peaksdb126['NAAF factor']
peaksdb126_NAAF['L-NAAF20'] = peaksdb126_NAAF['L'] * peaksdb126['NAAF factor']
peaksdb126_NAAF['M-NAAF20'] = peaksdb126_NAAF['M'] * peaksdb126['NAAF factor']
peaksdb126_NAAF['N-NAAF20'] = peaksdb126_NAAF['N'] * peaksdb126['NAAF factor']
peaksdb126_NAAF['P-NAAF20'] = peaksdb126_NAAF['P'] * peaksdb126['NAAF factor']
peaksdb126_NAAF['Q-NAAF20'] = peaksdb126_NAAF['Q'] * peaksdb126['NAAF factor']
peaksdb126_NAAF['R-NAAF20'] = peaksdb126_NAAF['R'] * peaksdb126['NAAF factor']
peaksdb126_NAAF['S-NAAF20'] = peaksdb126_NAAF['S'] * peaksdb126['NAAF factor']
peaksdb126_NAAF['T-NAAF20'] = peaksdb126_NAAF['T'] * peaksdb126['NAAF factor']
peaksdb126_NAAF['V-NAAF20'] = peaksdb126_NAAF['V'] * peaksdb126['NAAF factor']
peaksdb126_NAAF['W-NAAF20'] = peaksdb126_NAAF['W'] * peaksdb126['NAAF factor']
peaksdb126_NAAF['Y-NAAF20'] = peaksdb126_NAAF['Y'] * peaksdb126['NAAF factor']

# multiply the NAAF20 factor by the PTMs normalize its abundance by peak area and peptide length

peaksdb126_NAAF['ccarb-NAAF20'] = peaksdb126_NAAF['c-carb'] * peaksdb126_NAAF['NAAF factor']
peaksdb126_NAAF['moxid-NAAF20'] = peaksdb126_NAAF['m-oxid'] * peaksdb126_NAAF['NAAF factor']
peaksdb126_NAAF['ndeam-NAAF20'] = peaksdb126_NAAF['n-deam'] * peaksdb126_NAAF['NAAF factor']
peaksdb126_NAAF['qdeam-NAAF20'] = peaksdb126_NAAF['q-deam'] * peaksdb126_NAAF['NAAF factor']


# write the dataframe to a new csv
peaksdb126_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/126_MN_T00_GF_PDB/126B_MN_T00_GF_PDB_naaf.csv")

peaksdb126_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,R-NAAF20,S-NAAF20,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20
0,LPQVEGTGGDVQPSQDLVR,3.324576,0,0,2,1,0,3,0,0,...,3.324576,3.324576,3.324576,9.973727,0.0,0.000000,0.000000,0.0,0.0,0.0
1,VIGQNEAVDAVSNAIR,3.139695,3,0,1,1,0,1,0,2,...,3.139695,3.139695,0.000000,9.419086,0.0,0.000000,0.000000,0.0,0.0,0.0
2,SSGSSYPSLLQCLK,0.064304,0,1,0,0,0,1,0,0,...,0.000000,0.321519,0.000000,0.000000,0.0,0.064304,0.064304,0.0,0.0,0.0
3,LGEHNIDVLEGNEQFINAAK,0.000000,2,0,1,3,1,2,1,2,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
4,VTDAEIAEVLAR,0.659025,3,0,1,2,0,0,0,1,...,0.659025,0.000000,0.659025,1.318050,0.0,0.000000,0.000000,0.0,0.0,0.0


In [10]:
# made a new dataframe that contains the sums of NAAF normalized AAs for peaksdb126 peaksdb results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb126_NAAF['NAAF factor'].sum(),
        'A': peaksdb126_NAAF['A-NAAF20'].sum(),
        'C': peaksdb126_NAAF['C-NAAF20'].sum(),
        'D': peaksdb126_NAAF['D-NAAF20'].sum(),
        'E': peaksdb126_NAAF['E-NAAF20'].sum(),
        'F': peaksdb126_NAAF['F-NAAF20'].sum(),
        'G': peaksdb126_NAAF['G-NAAF20'].sum(),
        'H': peaksdb126_NAAF['H-NAAF20'].sum(),
        'I': peaksdb126_NAAF['I-NAAF20'].sum(),
        'K': peaksdb126_NAAF['K-NAAF20'].sum(),
        'L': peaksdb126_NAAF['L-NAAF20'].sum(),
        'M': peaksdb126_NAAF['M-NAAF20'].sum(),
        'N': peaksdb126_NAAF['N-NAAF20'].sum(),
        'P': peaksdb126_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb126_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb126_NAAF['R-NAAF20'].sum(),
        'S': peaksdb126_NAAF['S-NAAF20'].sum(),
        'T': peaksdb126_NAAF['T-NAAF20'].sum(),
        'V': peaksdb126_NAAF['V-NAAF20'].sum(),
        'W': peaksdb126_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb126_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb126_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb126_NAAF['moxid-NAAF20'].sum(),
        'n-deam': peaksdb126_NAAF['ndeam-NAAF20'].sum(),
        'q-deam': peaksdb126_NAAF['qdeam-NAAF20'].sum()
       }

totalpeaksdb126_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb126_NAAF['% C w/ carb.'] = totalpeaksdb126_NAAF['c-carb'] / totalpeaksdb126_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb126_NAAF['% M w/ oxid'] = totalpeaksdb126_NAAF['m-oxid'] / totalpeaksdb126_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb126_NAAF['% N w/ deam'] = totalpeaksdb126_NAAF['n-deam'] / totalpeaksdb126_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb126_NAAF['% Q w/ deam'] = totalpeaksdb126_NAAF['q-deam'] / totalpeaksdb126_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb126_NAAF['NAAF check'] = totalpeaksdb126_NAAF['NAAF'] / 201054.547872

# write modified dataframe to new txt file, same name + totals
totalpeaksdb126_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/126_MN_T00_GF_PDB/126B_MN_T00_GF_PDB_naaf_totals.csv")

totalpeaksdb126_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,163.445603,156.401491,0.145993,17.457509,21.15878,0.474177,48.774403,1.198067,24.663597,17.179784,...,0.386796,0.145993,0.011302,0.294437,0.507518,1.0,0.003417,0.005211,0.028406,0.000813
